In [432]:
from __future__ import absolute_import, print_function
import tweepy
import json
import os
import pandas as pd
from tweepy.streaming import StreamListener
import pprint as pp

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [433]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
consumer_token = os.getenv("access_token")
access_token_secret = os.getenv("access_token_secret")

In [434]:
def authorize_twitter_api():
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    return auth

In [435]:
api = tweepy.API(auth)

In [422]:
search_results = api.search(q='boss',tweet_mode = 'extended',count=1,lang = 'en')

In [ ]:
search_results

In [437]:
tweets_list = []
for tweet in tweepy.Cursor(api.search, q='bitcoin',tweet_mode = 'extended').items(5):
    tweets_list.append(tweet)

In [438]:
for i in range(len(tweets_list)):
    print (tweets_list[i].full_text)

RT @Rhythmtrader: The Federal Reserve had to loan banks the equivalent of 12,700,000 bitcoin last night to stabilize the financial system.
Buy #ARK/BTC 
 Market: Binance
 Buy Price: 0.00002512 
 Sell Price1: 0.00002525 +0.50%
 Sell Price2: 0.00002537 +1.00%
 Sell Price3: 0.00002550 +1.50%
 Sell Price4: 0.00002562 +2.00%
 Stop Loss: 0.00002437 -3.00%
 Ratio: 2:3 #Binance , #BTC , #Bitcoin , #CryptoCurrency , #Cr…
BINANCE
#BTC_ARK 
LONG
SL 3% #Binance , #BTC , #Bitcoin , #CryptoCurrency , #CryptoTrading , #CryptoSignals
@statusquont Send me an invoice when you are back there. Would be awesome to send some Bitcoin to you in Antarctica. I could imagine though that you would have limited internet.
Blockchain is creating a landscape of opportunity by replacing market friction with transparency, opening the door to a thriving global market. Which countries and industries are the ones to watch in 2020?

#globalmarket #bitcoin #btc #blockchainvancouver https://t.co/gBcGvz2ZpI


In [ ]:

# with open('tweet', 'w', encoding='utf-8') as f:
# json.dumps(tweet.text, indent=4)

In [ ]:
# for friend in tweepy.Cursor(api.friends).items():
#     # Process the friend here
#     process_friend(friend)

In [ ]:
pp = pprint.PrettyPrinter(depth=4)

#vars >>> OBJECT TO A DICTIONARY 
keys = vars(tweets_list[0]).keys()
print(keys)

In [ ]:
tweets_list[1].user


In [ ]:
tweets_columns = []

        
for i in range(len(tweets_list)):
        
        filtered_columns={
        'date':tweets_list[i].created_at,
        'user_id':tweets_list[i].user.id_str,
        'screen_name':tweets_list[i].user.screen_name,
        'text':tweets_list[i].full_text,
        'reply_to_screen_name':tweets_list[i].in_reply_to_screen_name,
        'reply_to_id':tweets_list[i].in_reply_to_user_id_str,
        'reply_to_status_id':tweets_list[i].in_reply_to_status_id
        }
        
        ##Becausee the below values are not present in all tweets, we must put them each in a try statement or an error will be thrown.
        try:
            filtered_columns.update({'user_mentions_screen_name':tweets_list[i].entities['user_mentions'][0]['screen_name']}) 
        except:
            filtered_columns.update({'user_mentions_screen_name':None})
        try:
            filtered_columns.update({'user_mentions_id':tweets_list[i].entities['user_mentions'][0]['id_str']})
        except:
            filtered_columns.update({'user_mentions_id':None})
        try:
            filtered_columns.update({'rt_status_screen_name':tweets_list[i].retweeted_status.user.screen_name}) 
        except:
            filtered_columns.update({'rt_status_screen_name':None})
        try:
            filtered_columns.update({'rt_status_id':tweets_list[i].retweeted_status.user.id_str})
        except:
            filtered_columns.update({'rt_status_id':None})
        try:
            filtered_columns.update({'language':tweets_list[i].lang}) 
        except:
            filtered_columns.update({'language':None}) 
        try:
            filtered_columns.update({'location':tweets_list[i].geo}) 
        except:
            filtered_columns.update({'location':None})
            
        try:
            filtered_columns.update({'verified': tweets_list[i].verified})
        except:
            filtered_columns.update({'verified':None})
        
        tweets_columns.append(filtered_columns)       
            
            

In [ ]:
tweets = pd.DataFrame(tweets_columns)

In [ ]:
tweets.head()

In [ ]:
tweets.set_index('date',inplace=True)


In [ ]:
tweets.info()


In [ ]:
tweets.info()

## Pickle Data## 

In [ ]:
# tweets.to_pickle('../output/tweets')




In [ ]:
def create_tweets_table(term_to_search):
    """
    This function open a connection with an already created database and creates a new table to
    store tweets related to a subject specified by the user
    """
    
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbnametwitter, user=usertwitter, password=passwordtwitter, host=hosttwitter, port=porttwitter)

    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, create two tables, users twitter and the corresponding table according to the selected topic
    cursor_twitter.execute("CREATE TABLE IF NOT EXISTS twitter_users (user_id VARCHAR PRIMARY KEY, user_name VARCHAR);")
    
    query_create = "CREATE TABLE IF NOT EXISTS %s (id SERIAL, created_at timestamp, tweet text NOT NULL, user_id VARCHAR, user_name VARCHAR, retweetstatus_user int, retweetstatus_name VARCHAR, PRIMARY KEY(id), FOREIGN KEY(user_id) REFERENCES twitter_users(user_id));" %("tweets_predict_"+term_to_search)
    cursor_twitter.execute(query_create)
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

In [ ]:
def store_tweets_in_table(term_to_search, created_at, tweet, user_id, user_name, retweetstatus_user, retweetstatus_name):
    """
    This function open a connection with an already created database and inserts into corresponding table 
    tweets related to the selected topic
    """
    
    #Connect to Twitter Database created in Postgres
    conn_twitter = psycopg2.connect(dbname=dbnametwitter, user=usertwitter, password=passwordtwitter, host=hosttwitter, port=porttwitter)

    #Create a cursor to perform database operations
    cursor_twitter = conn_twitter.cursor()

    #with the cursor now, insert tweet into table
    cursor_twitter.execute("INSERT INTO twitter_users (user_id, user_name) VALUES (%s, %s) ON CONFLICT(user_id) DO NOTHING;", (user_id, user_name))
    
    cursor_twitter.execute("INSERT INTO %s (created_at, tweet, user_id, user_name, retweetstatus_user, retweetstatus_name) VALUES (%%s, %%s, %%s, %%s, %%s, %%s);" %('tweets_predict_'+term_to_search), 
                           (created_at, tweet, user_id, user_name, retweetstatus_user, retweetstatus_name))
    
    #Commit changes
    conn_twitter.commit()
    
    #Close cursor and the connection
    cursor_twitter.close()
    conn_twitter.close()
    return

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    
    '''
    def on_status(self, status):
        print(status.text)
    '''
        
    def on_data(self, raw_data):

        try:
            global term_to_search
            
            data = json.loads(raw_data)            
            
            #Obtain all the variables to store in each column
            user_id = data['user']['id']
            user_name = data['user']['name']
            created_at = data['created_at']
            tweet = data['text']
            if data['retweeted_status']:
                retweetstatus_user = data['retweeted_status']['user']['id']
                retweetstatus_name = data['retweeted_status']['user']['name']
            else:
                retweetstatus_user = np.nan
                retweetstatus_name = ""
            
            #Store them in the corresponding table in the database
            store_tweets_in_table(term_to_search, created_at, tweet, user_id, user_name, retweetstatus_user, retweetstatus_name)        
            
        except Exception as e:
            print(e)
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [ ]:

# For realtime streaming
if __name__ == "__main__": 
    #Creates the table for storing the tweets
    term_to_search = "avengers"
    create_tweets_table(term_to_search)
    
    #Connect to the streaming twitter API
    api = tweepy.API(wait_on_rate_limit_notify=True)
    
    #Stream the tweets
    streamer = tweepy.Stream(auth=autorize_twitter_api(), listener=MyStreamListener(api=api))
    streamer.filter(languages=["en"], track=[term_to_search])